<a href="https://colab.research.google.com/github/A-K-ay/torrent_to_mega/blob/main/torrent_to_mega_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent To Mega

### Add Required User Inputs

In [ ]:
# Mega Credentials

megaEmail = "" #Email/Username
megaPassword = ""    #Password

# Torrent Magnet Link
torrentMagentLink = ""


### Import Dependencies


In [ ]:
!apt install python3-libtorrent

In [ ]:
!pip install mega.py

### Code to download torrent

In [ ]:
import libtorrent as lt
import time
import datetime

ses = lt.session()
ses.listen_on(6881, 6891)
params = {
    'save_path': '/torrents',
    'storage_mode': lt.storage_mode_t(2),
    'paused': False,
    'auto_managed': True,
    'duplicate_is_error': True}
print(torrentMagentLink)

handle = lt.add_magnet_uri(ses, torrentMagentLink, params)
ses.start_dht()

begin = time.time()
print(datetime.datetime.now())

print ('Downloading Metadata...')
while (not handle.has_metadata()):
    time.sleep(1)
print ('Got Metadata, Starting Torrent Download...')

torrentFolderName = handle.name()
print("Starting", handle.name())

while (handle.status().state != lt.torrent_status.seeding):
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata', \
            'downloading', 'finished', 'seeding', 'allocating']
    print ('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
            (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
            s.num_peers, state_str[s.state]))
    time.sleep(5)

end = time.time()
print(handle.name(), "COMPLETE")

print("Elapsed Time: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")

print(datetime.datetime.now())

### Create a Zip of the downloaded torrent

In [ ]:
import shutil
begin = time.time()
shutil.make_archive(torrentFolderName, 'zip', "/torrents/"+torrentFolderName)
end = time.time()
print("Compression Successful")
print("Elapsed Time: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")


### Upload Zip To Mega

In [ ]:
from mega import Mega
mega = Mega()

# Login
# Add your credentials here

m = mega.login(megaEmail,megaPassword)


begin = time.time()

# Upload File
file = m.upload('/content/'+torrentFolderName+".zip")
end = time.time()
print(torrentFolderName, "COMPLETE")

print("Elapsed Time: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")
